In [171]:
from fpdf import FPDF
import os
import yaml
import pandas as pd
from PIL import Image

In [172]:
def add_image_to_pdf(pdf, firstChart_src, firstChart_title, firstChart_source):
    # Set title font and add title text
    pdf.set_font('OpenSans', 'B', 14)
    pdf.cell(0, 10, firstChart_title, 0, 1, 'C')  # Centered title
    pdf.ln(2)  # Add space below the title

    # Load the image to get its dimensions
    image = Image.open(firstChart_src)
    original_width, original_height = image.size

    # Define the desired width and calculate proportional height
    desired_width = 200
    aspect_ratio = original_height / original_width
    calculated_height = desired_width * aspect_ratio

    # Center the image horizontally
    x_position = (pdf.w - desired_width) / 2

    # Insert the image
    pdf.image(firstChart_src, x=x_position, y=pdf.get_y(), w=desired_width, h=calculated_height)
    pdf.ln(calculated_height + 0)  # Move cursor below the image

    # Add the source text aligned to the right
    pdf.set_font('OpenSans', 'I', 9)
    pdf.cell(0, 10, f"Source: {firstChart_source}", 0, 0, 'R')  # Right-aligned source text
    pdf.ln(10)  # Optional: Add extra space below the source text

In [173]:
def add_table_to_pdf(pdf, table_src, table_source):
    """
    Adds a table to the PDF with headers, row data, and custom formatting
    for negative numbers and thousands separators.
    """
    
    # Importing Table
    table_page = pd.read_csv(table_src, sep = ",")
    # Set the first column as the index
    table_page.index = table_page.iloc[:, 0]
    
    # Drop the first column now that it's set as the index
    table_page = table_page.drop(table_page.columns[0], axis=1)
    
    # Headers
    headers = [' '] + table_page.columns.tolist()
    col_widths = [65] + [25] * (len(headers) - 1)
    
    # Add header row with bottom border
    pdf.set_font('OpenSans', 'B', 10)
    for i, header in enumerate(headers):
        pdf.cell(col_widths[i], 10, header, border="B", align="C")
    pdf.ln()  # Move to the next line after the header row

    # Row Data
    for index, row in table_page.iterrows():
        # Row Name (Index)
        pdf.set_font('OpenSans', 'I', 9)
        pdf.cell(col_widths[0], 10, str(index), "", 0, align="C")
        
        # Row Data
        for i, item in enumerate(row):
            # Format the item if it is a number
            if isinstance(item, (int, float)):
                # Format with commas for thousands separator
                item_str = f"{item:,.0f}" if isinstance(item, int) else f"{item:,.2f}"
                
                # Set text color to red if negative
                if item < 0:
                    pdf.set_text_color(255, 0, 0)  # Red for negative numbers
                else:
                    pdf.set_text_color(0, 0, 0)    # Black for positive numbers
            else:
                item_str = str(item)  # Leave as-is if not a number
                pdf.set_text_color(0, 0, 0)      # Black for non-numeric data

            # Print the cell
            pdf.set_font('OpenSans', '', 9)
            pdf.cell(col_widths[i + 1], 10, item_str, "", 0, 'C')  # No border

        # Reset to black for the next row
        pdf.set_text_color(0, 0, 0)
        pdf.ln()  # Move to the next line after the row

    # Draw bottom border for the entire last row
    for i in range(len(headers)):
        pdf.cell(col_widths[i], 0, '', 'B', 0, 'C')
    pdf.ln()
    
    pdf.ln()  # Move to the next line after the last row

    # Add "Fonte: " text aligned to the right
    pdf.set_font('OpenSans', 'I', 9)
    pdf.cell(0, 10, f"Source: {table_source}", 0, 0, 'R')  # 0 width to stretch across the page, aligned right

In [174]:
# Create a class that inherits from FPDF
class PDF(FPDF):
    def header(self):
        # Add image from image folder
        image_folder = "./static/images/Reports Headers.png"
        self.image(image_folder, 0, 0, 210)

    def footer(self):
        # Go to 1.5 cm from bottom
        self.set_y(-15)
        # Select Arial italic 8
        self.set_font('Arial', 'I', 8)
        # Page number
        self.cell(0, 10, f'Page {self.page_no()}', 0, 0, 'C')
        
def generate_report(output_file):
    # Create a PDF object
    pdf = PDF("P", "mm", "A4")

    # Add Open Sans font (adjust the path to your font files)
    fonts_folder = "./static/fonts/Open_Sans/static/"
    pdf.add_font("OpenSans", "", os.path.join(fonts_folder, "OpenSans-Regular.ttf"), uni=True)
    pdf.add_font("OpenSans", "B", os.path.join(fonts_folder, "OpenSans-Bold.ttf"), uni=True)
    pdf.add_font("OpenSans", "I", os.path.join(fonts_folder, "OpenSans-Italic.ttf"), uni=True)
    
    
    # Load the YAML file
    with open('./page_objects.yaml', 'r') as file:
        data = yaml.safe_load(file)
        
        # Table Data
        table_title = data['crudeOilPage']['table_title']
        table_src = data['crudeOilPage']['table_src']
        table_source = data['crudeOilPage']['table_source']
        
        # Table Data
        firstChart_title = data['crudeOilPage']['firstChart_title']
        firstChart_src = data['crudeOilPage']['firstChart_src']
        firstChart_source = data['crudeOilPage']['firstChart_source']
    
    # Add a page
    pdf.add_page()
    
    # Define the text content
    text_page = """
  Commercial crude oil inventories rose by 2.1 million barrels, marking a 1.16% increase over the past 30 days, although they remained 1.86% lower than the same period last year. Strategic reserves recorded a 10.23% annual increase, while total U.S. crude oil stocks, including the Strategic Petroleum Reserve (SPR), posted a 3.54% year-over-year rise.
  """
    
    # Set the Y position for the body content
    pdf.set_y(45)
    
    # Set title font and add title text
    pdf.set_font('OpenSans', 'B', 16)
    pdf.cell(0, 0, "Resume", 0, 0, 'l')  # Centered title
    pdf.ln(5)  # Add space below the title
    
    pdf.set_font('OpenSans', '', 9)
    
    # Add the multi-line text using multi_cell
    pdf.multi_cell(0, 5, text_page)
        
    # Set the Y position for body content
    pdf.ln(5)

    # Page Title
    pdf.set_font('OpenSans', 'B', 14)
    pdf.cell(0, 10, table_title)
    
    # Table
    pdf.ln(10)
    add_table_to_pdf(pdf, table_src, table_source)
    
    # First Chart
    pdf.ln(20)
    add_image_to_pdf(pdf, firstChart_src, firstChart_title, firstChart_source)
    

    # Save the PDF to a file
    pdf.output(output_file)


In [175]:
# Save the PDF to a file
from datetime import datetime
today = datetime.now().strftime("%Y %m %d")
file_name = f'Petroleum & Other Liquids Stocks - {today}.pdf'
output_file = os.path.join("../report/", file_name )  # Full path for the PDF

generate_report(output_file)